In [53]:
import pandas as pd
import numpy as np
import os

from sklearn import metrics

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
import matplotlib.pyplot as plt

import sklearn.preprocessing as pp
from sklearn.neighbors import KNeighborsClassifier


In [74]:
combined_df = pd.read_csv('merged.csv')

In [55]:
temp = combined_df.dropna()

In [64]:
temp

,AGENCY IDENTIFIER,BIAS MOTIVATION,CITY,JUDICIAL DISTRICT,LOCATION,POPULATION,QUARTER,REGION,STATE,YEAR
0,AK0010100,Anti-Race,ANCHORAGE,020A,Residence/home,260900,Jul to Sep,West,AK,2000
1,AK0010100,Anti-Race,ANCHORAGE,020A,Residence/home,260900,Jul to Sep,West,AK,2000
2,AK0010100,Anti-Race,ANCHORAGE,020A,Highway/road/alley,260900,Jan to Mar,West,AK,2000
3,AK0010100,Anti-Race,ANCHORAGE,020A,Residence/home,260900,Jan to Mar,West,AK,2000
4,AR0080300,Anti-Race,GREEN FOREST,035W,Residence/home,2785,Jul to Sep,South,AR,2000
...,...,...,...,...,...,...,...,...,...,...
128727,WV0490100,Anti-Race,BUCKHANNON,430N,Bar/nightclub,5662,Oct to Dec,South,WV,2016
128728,WV0540100,Anti-Sexual Orientation,PARKERSBURG,435S,Highway/road/alley,30913,Jan to Mar,South,WV,2016
128729,WY0030100,Anti-Race,GILLETTE,450A,School--elementary/secondary,33218,Apr to Jun,West,WY,2016
128730,WY0030100,Anti-Race,GILLETTE,450A,School--elementary/secondary,33218,Apr to Jun,West,WY,2016


In [75]:
temp.YEAR.unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2010, 2011, 2012,
       2013, 2014, 2015, 2016])

In [57]:
class StdScalerByGroup(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        """
        :Example:
        >>> cols = {'g': ['A', 'A', 'B', 'B'], 'c1': [1, 2, 2, 2], 'c2': [3, 1, 2, 0]}
        >>> X = pd.DataFrame(cols)
        >>> std = StdScalerByGroup().fit(X)
        >>> std.grps_ is not None
        True
        """
        # X may not be a pandas dataframe (e.g. a np.array)
        df = pd.DataFrame(X)
        mean_df = df.groupby(df.columns[0]).mean()
        std_df = df.groupby(df.columns[0]).std()
        
        # A dictionary of means/standard-deviations for each column, for each group.
        self.grps_ = [mean_df, std_df]
        return self

    def transform(self, X, y=None):
        """
        :Example:
        >>> cols = {'g': ['A', 'A', 'B', 'B'], 'c1': [1, 2, 3, 4], 'c2': [1, 2, 3, 4]}
        >>> X = pd.DataFrame(cols)
        >>> std = StdScalerByGroup().fit(X)
        >>> out = std.transform(X)
        >>> out.shape == (4, 2)
        True
        >>> np.isclose(out.abs(), 0.707107, atol=0.001).all().all()
        True
        """
        try:
            getattr(self, "grps_")
        except AttributeError:
            raise RuntimeError("You must fit the transformer before tranforming the data!")
        
        # X may not be a dataframe (e.g. np.array)
        df = pd.DataFrame(X)
        
        grouped_mean = self.grps_[0]
        grouped_std = self.grps_[1]
        numerator = (df.set_index(df.columns[0])-grouped_mean.reindex(df[df.columns[0]])).reset_index()
        z_score = (numerator.set_index(df.columns[0])/grouped_std.reindex(df[df.columns[0]])).reset_index()
        return z_score.set_index(z_score.columns[0])

In [58]:
x_value = temp.drop(columns=['BIAS MOTIVATION'])
y_value = temp['BIAS MOTIVATION']
x_train, x_test, y_train, y_test = train_test_split(x_value, y_value, test_size=0.25)

In [59]:
def titanic_model(titanic):
    """
    :Example:
    >>> fp = os.path.join('data', 'titanic.csv')
    >>> data = pd.read_csv(fp)
    >>> pl = titanic_model(data)
    >>> isinstance(pl, Pipeline)
    True
    >>> from sklearn.base import BaseEstimator
    >>> isinstance(pl.steps[-1][-1], BaseEstimator)
    True
    >>> preds = pl.predict(data.drop('Survived', axis=1))
    >>> ((preds == 0)|(preds == 1)).all()
    True
    """    
    def prefix(col):
        col = col.iloc[:,0]
        return np.array(col.str.split().str[:1].str[0]).reshape(-1,1)
    
    #name_feat = ['Name']
    #name_tran = Pipeline([
        #('title', FunctionTransformer(prefix, validate=False)),
        #('onehot', OneHotEncoder(handle_unknown='ignore'))
    #])

#     p_feat = ['POPULATION']
#     p_tran = Pipeline([
#        ('standard unit', pp.StandardScaler())
#     ])

    cat_feat = ['AGENCY IDENTIFIER', 'CITY', 'JUDICIAL DISTRICT',
       'LOCATION', 'POPULATION', 'QUARTER', 'REGION', 'STATE', 'YEAR']
    cat_tran = Pipeline([
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # preprocessing pipeline (put them together)
    pre = ColumnTransformer(transformers=[('cat', cat_tran, cat_feat)], remainder='passthrough')

    comb = Pipeline([('preprocessor', pre), ('rfc', RandomForestClassifier(n_estimators=180,min_samples_leaf=3))])
    comb.fit(x_train,y_train)
    return comb

In [60]:
p1 = titanic_model(temp)

In [61]:
preds = p1.predict(x_test)

In [62]:
print(metrics.accuracy_score(y_test, preds))

0.6716124809077024


In [70]:
pd.Series(preds).value_counts()

Anti-Race                  28996
Anti-Religion               2718
Anti-Sexual Orientation      341
Anti-Disability               26
dtype: int64

In [40]:
def titanic_model(titanic):
    """
    :Example:
    >>> fp = os.path.join('data', 'titanic.csv')
    >>> data = pd.read_csv(fp)
    >>> pl = titanic_model(data)
    >>> isinstance(pl, Pipeline)
    True
    >>> from sklearn.base import BaseEstimator
    >>> isinstance(pl.steps[-1][-1], BaseEstimator)
    True
    >>> preds = pl.predict(data.drop('Survived', axis=1))
    >>> ((preds == 0)|(preds == 1)).all()
    True
    """    
    def prefix(col):
        col = col.iloc[:,0]
        return np.array(col.str.split().str[:1].str[0]).reshape(-1,1)
    
    #name_feat = ['Name']
    #name_tran = Pipeline([
        #('title', FunctionTransformer(prefix, validate=False)),
        #('onehot', OneHotEncoder(handle_unknown='ignore'))
    #])

#     p_feat = ['POPULATION']
#     p_tran = Pipeline([
#        ('standard unit', pp.StandardScaler())
#     ])

    cat_feat = ['AGENCY IDENTIFIER', 'CITY', 'JUDICIAL DISTRICT',
       'LOCATION', 'POPULATION', 'QUARTER', 'REGION', 'STATE', 'YEAR']
    cat_tran = Pipeline([
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # preprocessing pipeline (put them together)
    pre = ColumnTransformer(transformers=[('cat', cat_tran, cat_feat)], remainder='passthrough')

    comb = Pipeline([('preprocessor', pre), ('rfc', KNeighborsClassifier())])
    comb.fit(x_train,y_train)
    return comb

In [44]:
p1 = titanic_model(temp1)

In [45]:
preds = p1.predict(x_test)

In [46]:
print(metrics.accuracy_score(y_test, preds))

0.6588323306630093


In [19]:
from sklearn.naive_bayes import GaussianNB

In [20]:
def titanic_model(titanic):
    """
    :Example:
    >>> fp = os.path.join('data', 'titanic.csv')
    >>> data = pd.read_csv(fp)
    >>> pl = titanic_model(data)
    >>> isinstance(pl, Pipeline)
    True
    >>> from sklearn.base import BaseEstimator
    >>> isinstance(pl.steps[-1][-1], BaseEstimator)
    True
    >>> preds = pl.predict(data.drop('Survived', axis=1))
    >>> ((preds == 0)|(preds == 1)).all()
    True
    """    
    def prefix(col):
        col = col.iloc[:,0]
        return np.array(col.str.split().str[:1].str[0]).reshape(-1,1)
    
    #name_feat = ['Name']
    #name_tran = Pipeline([
        #('title', FunctionTransformer(prefix, validate=False)),
        #('onehot', OneHotEncoder(handle_unknown='ignore'))
    #])

#     p_feat = ['POPULATION']
#     p_tran = Pipeline([
#        ('standard unit', pp.StandardScaler())
#     ])

    cat_feat = ['AGENCY IDENTIFIER', 'CITY', 'JUDICIAL DISTRICT',
       'LOCATION', 'POPULATION', 'QUARTER', 'REGION', 'STATE', 'YEAR']
    cat_tran = Pipeline([
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # preprocessing pipeline (put them together)
    pre = ColumnTransformer(transformers=[('cat', cat_tran, cat_feat)], remainder='passthrough')

    comb = Pipeline([('preprocessor', pre), ('rfc', GaussianNB())])
    comb.fit(x_train,y_train)
    return comb